# Actuarial Standards of Practice (ASOP) Q&A Machine using Retrieval Augmented Generation (RAG)
This project aims to create a Retrieval-Augmented Generation (RAG) process for actuaries to ask questions on a set of Actuarial Standards of Practice (ASOP) documents. The RAG process utilizes the power of the Large Language Model (LLM) to provide answers to questions on ASOPs.

However, RAG is not without challenges, i.e., hallucination and inaccuracy. This code allows verifiability by providing the context it used to arrive at those answers. This process enables actuaries to validate the information provided by the LLM, empowering them to make informed decisions. By combining the capabilities of LLM with verifiability, this code offers actuaries a robust tool to leverage LLM technology effectively and extract maximum value.

The current example uses either OpenAI's GPT 3.5 turbo or a local LLM. Using a local LLM can address potential data privacy or security concerns.

View license or further information about the local models used:
- Solar 10.7B Instruct: [cc-by-nc-4.0](https://huggingface.co/upstage/SOLAR-10.7B-Instruct-v1.0) (non-commercial use)
- Mistral 7B Instruct: [Apache License 2.0](https://ollama.com/library/mistral/blobs/sha256:43070e2d4e532684de521b885f385d0841030efa2b1a20bafb76133a5e1379c1)
- [GPT4All embedding model](https://python.langchain.com/docs/integrations/text_embedding/gpt4all)

# 1. Initial Setup
This setup includes loading environment variables from a `.env` file, setting the required environment variables, and importing the necessary modules for further processing. It ensures that the code has access to the required APIs and functions for the subsequent tasks.


In [35]:
# Initial set up
from dotenv import load_dotenv
import os

# Load the variables from .env file and set the API key (or user may manually set the API key)
load_dotenv()  # This loads the variables from .env (not part of repo)
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
#os.environ["LANGCHAIN_TRACING_V2"] = "true" # use when you want to debug or monitor the performance of your langchain applications
#os.environ["LANGCHAIN_API_KEY"] = os.getenv('LANGCHAIN_API_KEY') # use when accessing cloud-based language models or services that langchain integrates with

# Import the necessary modules
import bs4
from langchain import hub
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.llms import Ollama
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableParallel # for RAG with source
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from IPython.display import display, Markdown, Latex
import glob
import chromadb

In [36]:
# Change to False if using local models instead of OpenAI models
use_OpenAI = False

if use_OpenAI:
    embeddings_model = OpenAIEmbeddings()
    db_directory = "../data/chroma_db1"
    llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125", 
                     temperature=0) # context window size 16k for GPT 3.5 Turbo

else: #Open source models used here are for illustration and educational purposes
    embeddings_model = GPT4AllEmbeddings()
    db_directory = "../data/chroma_db2"
    # define a local large language model for the augmented generation
    # Ollama is one way to easily run inference
    #llm = Ollama(model="solar:10.7b-instruct-v1-q5_K_M")
    llm = Ollama(model="mistral:instruct")
    

bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522


# 2. Load PDF Files and Convert to a Vector DB
1. Create a function to load and extract text from PDF files in a specified folder. It defines a function called `load_pdfs_from_folder()` that takes a folder path as input and returns a list of extracted text documents from the PDF files in that folder.

2. In the example, the folder path `../data/ASOP` is used, but you can modify it to point to your desired folder.

3. By calling the `load_pdfs_from_folder()` function with the folder path, the code loads the PDF files, extracts the text using the PyPDFLoader, and stores the extracted text documents in the `docs` list.

4. After loading and extracting the text, a `RecursiveCharacterTextSplitter` object is created with specific parameters for chunking the documents. The `split_documents()` method is then used to split the documents into smaller chunks based on the specified parameters.

5. Finally, a Chroma vectorstore is created from the document splits. The vectorstore uses the defined embedding model for embedding the chunks and is saved to the predefined directory.

In [37]:
# Run only when the DB directory is empty
if not os.path.exists(db_directory) or not os.listdir(db_directory):
    # Define a function to load and extract text from PDFs in a folder
    def load_pdfs_from_folder(folder_path):
        # Get a list of PDF files in the specified folder
        pdf_files = glob.glob(f"{folder_path}/*.pdf")
        docs = []
        for pdf_file in pdf_files:
            # Load the PDF file using the PyPDFLoader
            loader = PyPDFLoader(pdf_file) 
            # Extract the text from the PDF and add it to the docs list
            docs.extend(loader.load())
        return docs
    
    # Example folder path
    folder_path = '../data/ASOP'
    
    # Call the function to load and extract text from PDFs in the specified folder
    docs = load_pdfs_from_folder(folder_path)
    
    # Create a text splitter object with specified parameters
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000, 
        chunk_overlap=200,
        length_function=len,)
    
    # Split the documents into chunks using the text splitter
    splits = text_splitter.split_documents(docs)
    
    # Create a Chroma vector database from the document splits, using OpenAIEmbeddings for embedding
    vectorstore = Chroma.from_documents(documents=splits, 
                                        embedding=embeddings_model, 
                                        persist_directory=db_directory)

# 3. Retrieve from the Vector DB 

In [38]:
# Get a Chroma vector database with specified parameters
vectorstore = Chroma(embedding_function=embeddings_model, 
                     persist_directory=db_directory)

In [55]:
## Retrieve and RAG chain

# Create a retriever using the vector database as the search source
retriever = vectorstore.as_retriever(search_type="mmr", 
                                     search_kwargs={'k': 6, 'lambda_mult': 0.35}) 
# Use MMR (Maximum Marginal Relevance) to find a set of documents that are both similar to the input query and diverse among themselves
# Increase the number of documents to get, and increase diversity (lambda mult 0.5 being default, 0 being the most diverse, 1 being the least)

# Load the RAG (Retrieval-Augmented Generation) prompt
prompt = hub.pull("rlm/rag-prompt")

# Define a function to format the documents with their sources and pages
def format_docs_with_sources(docs):
    formatted_docs = "\n\n".join(doc.page_content for doc in docs)
    sources_pages = "\n".join(f"{doc.metadata['source']} (Page {doc.metadata['page'] + 1})" for doc in docs)
    # Added 1 to the page number assuming 'page' starts at 0 and we want to present it in a user-friendly way

    return f"Documents:\n{formatted_docs}\n\nSources and Pages:\n{sources_pages}"

# Create a RAG chain using the formatted documents as the context
rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs_with_sources(x["context"])))
    | prompt
    | llm
    | StrOutputParser()
)

# Create a parallel chain for retrieving and generating answers
rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

# 4. Generate Q&A Function

In [56]:
def generate_output():
    # Prompt the user for a question on ASOP
    usr_input = input("What is your question on ASOP?: ")

    # Invoke the RAG chain with the user input as the question
    output = rag_chain_with_source.invoke(usr_input)

    # Generate the Markdown output with the question, answer, and context
    markdown_output = "### Question\n{}\n\n### Answer\n{}\n\n### Context\n".format(output['question'], output['answer'])

    last_page_content = None  # Variable to store the last page content
    i = 1 # Source indicator

    # Iterate over the context documents to format and include them in the output
    for doc in output['context']:
        current_page_content = doc.page_content.replace('\n', '  \n')  # Get the current page content
        
        # Check if the current content is different from the last one
        if current_page_content != last_page_content:
            markdown_output += "- **Source {}**: {}, page {}:\n\n{}\n".format(i, doc.metadata['source'], doc.metadata['page'], current_page_content)
            i = i + 1
        last_page_content = current_page_content  # Update the last page content
    
    # Display the Markdown output
    display(Markdown(markdown_output))

# Example questions related to ASOPs
- explain ASOP No. 14
- How are expenses relfected in cash flow testing based on ASOP No. 22?
- What is catastrophe risk?
- When do I update assumptions?
- What should I do when I do not have credible data to develop non-economic assumptions?

In [57]:
generate_output()

What is your question on ASOP?:  Explain ASOP No. 14


### Question
Explain ASOP No. 14

### Answer
 ASOP No. 14, previously known as Section 3.14 of the ASB's Actuarial Standards of Practice, is related to cash flow testing for life and health insurance companies. It was incorporated into other ASOPs like ASOP No. 7 and ASOP No. 22 in 2001. The standard was repealed for work performed on or after April 15, 2002, and its current version is part of ASOP No. 38. The Catastrophe Modeling Task Force updated the exposure draft of ASOP No. 38 to maintain consistency with other recent ASOPs. (Sources: ASOP No. 11, ASOP No. 41, ASOP No. 56, ASOP No. 51, and ASOP No. 14)

### Context
- **Source 1**: ../data/ASOP/asop011_199.pdf, page 30:

after ASOP No. 11 was initially exposed.    
Section 3.14, Reliance on Experts (now section 3.16, Reliance on the Expertise of Others)    
Comment   
   
   
Response  One commentator said that this section appears to have been drawn from ASOP No. 56, and   
suggested deleting duplicative language and adding a reference to ASOP No. 56 instead.   
   
The reviewers believe the guidance is  not limited to modeling and made no change.
- **Source 2**: ../data/ASOP/asop041_120.pdf, page 26:

address the concerns of the commenta tor. The revised section 4.4 is part   
of the ASB initiative to move all su bstantive guidance on deviation into   
ASOP No. 41 (and thus achieve consis tency across ASOPs.) Part of this   
initiative is to clarify that “devia tion” means deviating from the guidance   
of an ASOP. Compliance with th e ASOP is still possible through   
adequate disclosure.
- **Source 3**: ../data/ASOP/asop056_195.pdf, page 24:

ASOP No. 56—Doc. No. 195   
   
   
 18Comment    
    
   
Response One commentator noted that section 2.10, Output, only mentions parameters as output that might be   
used as input to other models, while different sections of the proposed ASOP also mention data and   
assumptions as possible model outputs that can be used as input  to other models.   
   
The reviewers agree, made cha nges to the definitions of “assump tion,” “parameter,” “input,” and   
“output,” and removed references to “parameter” within section 3 of the ASOP to improve clarit  
y.   
Comment    
   
   
Response One commentator suggested elimin ating “qualitative criteria on which decisions could be made,” which   
is vague and may include unintended application of the ASOP.   
   
The reviewers believe the guidance is appropriate and therefore  made no change in response to this   
commen t.   
Section 2.11, Ov erfitting (now section 2.12)   
Comment
- **Source 4**: ../data/ASOP/asop051_188.pdf, page 5:

ASOP No. 51—September 2017   
   
   
 viThe Pension Committee thanks former committee chairperson Mita D. Drazilov and former   
committee members Fiona E. Liston, Mitchell I. Serota, Judy K. Stromback, and Virginia C.   
Wentz for their assistance with drafting this ASOP.  The ASB voted in September 2017 to adopt this standard.
- **Source 5**: ../data/ASOP/asop014_082.pdf, page 1:

and Health Insurance Companies , to provide guidance in determining whether or not   
to do cash flow testing in forming a professional opinion or recommendation.    
   
As part of the project to look at all cash flow testing standards of p ractice, a task force   
of the ASB’s Life Committee reviewed ASOP No. 7 (titled, as of September 2001,   
Analysis of Life, Health, or Property/Casualty Insurer Cash Flows ), ASOP No. 14   
(When to do Cash Flow Testing for Life and Health Insurance Companies ), and    
ASOP No. 22 (titled, as of September 2001, Statements of Opinion Based on Asset   
Adequacy Analysis by Actuaries for Life or Health Insurers ). Relevant portions of   
ASOP No. 14 were incorporated within the 2001 revisions of ASOP No. 7 and   
ASOP No. 22.    
   
At its September 2001 meeting, the ASB voted to adopt the revised ASOP No. 7 and   
ASOP No. 22 and to repeal ASOP No. 14.    
   
ASOP No. 14 is repealed for any work performed on or after April 15, 2002.
- **Source 6**: ../data/ASOP/asop038_201.pdf, page 3:

2019. Changes have been made to this exposure draft of ASOP No. 38 to be consistent with ASOP   
No. 56 and other recent ASOPs.   
   
The exposure draft of this revision of ASOP No. 38 was the work of the Catastrophe Modeling   
Task Force, whose membership has experience in life insurance, health insurance,   
property/casualty insurance, and enterprise risk management.


# 5. References
- https://www.actuarialstandardsboard.org/standards-of-practice/
- https://python.langchain.com/docs/use_cases/question_answering/quickstart
- https://python.langchain.com/docs/use_cases/question_answering/sources
- https://python.langchain.com/docs/integrations/text_embedding/
- https://python.langchain.com/docs/integrations/vectorstores/chroma
- https://docs.gpt4all.io/gpt4all_python_embedding.html#gpt4all.gpt4all.Embed4All
- https://chat.langchain.com/
- https://api.python.langchain.com/en/latest/vectorstores/langchain_community.vectorstores.chroma.Chroma.html

In [42]:
vectorstore._collection.count() # Number of collections

4754

In [47]:
vectorstore._collection.peek(1) # See what the first document of the vectorstore looks like

{'ids': ['8ee4f5ce-ca7b-11ee-947d-0e2e3f1e4b98'],
 'embeddings': [[0.01355269830673933,
   -0.01566288247704506,
   -0.0006177255418151617,
   0.005909645929932594,
   -0.03331739827990532,
   0.036120831966400146,
   -0.03325234353542328,
   0.08637803047895432,
   -0.047106821089982986,
   0.0415164940059185,
   0.10191214084625244,
   0.057127419859170914,
   0.0034442120231688023,
   0.051842473447322845,
   -0.007184958551079035,
   -0.05212683230638504,
   -0.012089588679373264,
   -0.026330195367336273,
   -0.007874737493693829,
   0.11539057642221451,
   0.03343813493847847,
   0.06662746518850327,
   0.0026790921110659838,
   -0.022108571603894234,
   0.03890356421470642,
   -0.07225413620471954,
   0.000745304103475064,
   -0.003962039016187191,
   0.01898397132754326,
   -0.03898923099040985,
   -0.0122222900390625,
   0.0885823592543602,
   0.0653577446937561,
   -0.02425547130405903,
   0.04874216020107269,
   -0.05872773751616478,
   -0.03356226161122322,
   -0.0021503255